# Solver Comparison: Vermeulen, Södergård, and Zakharov

This notebook compares three mechanistic free testosterone (FT) solvers:

1. **Vermeulen (1999)** - The reference standard using cubic solver
2. **Södergård (1982)** - Alternative binding constants
3. **Zakharov (2015)** - Allosteric cooperative binding model

We compare solver outputs across a physiological range of total testosterone (TT) and SHBG values.

In [ ]:
import sys
from pathlib import Path

# Add project root to path for imports
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import numpy as np
import matplotlib.pyplot as plt
from freeT.models import calc_ft_vermeulen, calc_ft_sodergard, calc_ft_zakharov

print("Solvers imported successfully!")

## 1. Define Synthetic Grid

Create a grid of physiological values:
- **TT**: 5-30 nmol/L (typical male range)
- **SHBG**: 10-80 nmol/L (normal range)
- **Albumin**: Fixed at 43 g/L (population mean)

In [ ]:
# Define parameter ranges
tt_range = np.linspace(5, 30, 26)   # TT: 5-30 nmol/L, 26 points
shbg_range = np.linspace(10, 80, 15) # SHBG: 10-80 nmol/L, 15 points
alb_fixed = 43.0  # g/L (population mean)

print(f"TT range: {tt_range[0]:.1f} - {tt_range[-1]:.1f} nmol/L ({len(tt_range)} points)")
print(f"SHBG range: {shbg_range[0]:.1f} - {shbg_range[-1]:.1f} nmol/L ({len(shbg_range)} points)")
print(f"Albumin: {alb_fixed} g/L (fixed)")
print(f"Grid size: {len(tt_range) * len(shbg_range)} calculations per solver")

## 2. Calculate FT for All Solvers

In [ ]:
# Initialize result arrays
results = {
    'tt': [],
    'shbg': [],
    'ft_vermeulen': [],
    'ft_sodergard': [],
    'ft_zakharov': []
}

# Calculate FT for each TT/SHBG combination
for tt in tt_range:
    for shbg in shbg_range:
        results['tt'].append(tt)
        results['shbg'].append(shbg)
        results['ft_vermeulen'].append(calc_ft_vermeulen(tt, shbg, alb_fixed))
        results['ft_sodergard'].append(calc_ft_sodergard(tt, shbg, alb_fixed))
        results['ft_zakharov'].append(calc_ft_zakharov(tt, shbg, alb_fixed))

# Convert to numpy arrays
for key in results:
    results[key] = np.array(results[key])

print(f"Calculated {len(results['tt'])} FT values for each solver")
print(f"\nVermeulen FT range: {results['ft_vermeulen'].min():.3f} - {results['ft_vermeulen'].max():.3f} nmol/L")
print(f"Södergård FT range: {results['ft_sodergard'].min():.3f} - {results['ft_sodergard'].max():.3f} nmol/L")
print(f"Zakharov FT range:  {results['ft_zakharov'].min():.3f} - {results['ft_zakharov'].max():.3f} nmol/L")

## 3. Visualize Solver Comparisons

### 3.1 FT vs TT at Different SHBG Levels

In [ ]:
# Plot FT vs TT for selected SHBG values
fig, axes = plt.subplots(1, 3, figsize=(14, 4), sharey=True)

shbg_levels = [20, 40, 60]  # Low, medium, high SHBG
colors = {'Vermeulen': '#2E86AB', 'Södergård': '#A23B72', 'Zakharov': '#F18F01'}

for ax, target_shbg in zip(axes, shbg_levels):
    # Find closest SHBG in our range
    closest_shbg = shbg_range[np.argmin(np.abs(shbg_range - target_shbg))]
    mask = results['shbg'] == closest_shbg
    
    ax.plot(results['tt'][mask], results['ft_vermeulen'][mask], 
            'o-', color=colors['Vermeulen'], label='Vermeulen', markersize=4)
    ax.plot(results['tt'][mask], results['ft_sodergard'][mask], 
            's--', color=colors['Södergård'], label='Södergård', markersize=4)
    ax.plot(results['tt'][mask], results['ft_zakharov'][mask], 
            '^:', color=colors['Zakharov'], label='Zakharov', markersize=4)
    
    ax.set_xlabel('Total Testosterone (nmol/L)')
    ax.set_title(f'SHBG = {closest_shbg:.0f} nmol/L')
    ax.grid(alpha=0.3)
    ax.legend(fontsize=8)

axes[0].set_ylabel('Free Testosterone (nmol/L)')
fig.suptitle('Free Testosterone vs Total Testosterone at Different SHBG Levels', fontsize=12)
plt.tight_layout()
plt.show()

### 3.2 Solver Agreement: Scatter Plots

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Vermeulen vs Södergård
ax = axes[0]
ax.scatter(results['ft_vermeulen'], results['ft_sodergard'], alpha=0.5, s=15, c='#2E86AB')
lims = [0, max(results['ft_vermeulen'].max(), results['ft_sodergard'].max()) * 1.05]
ax.plot(lims, lims, 'k--', alpha=0.5, label='Identity')
ax.set_xlabel('Vermeulen FT (nmol/L)')
ax.set_ylabel('Södergård FT (nmol/L)')
ax.set_title('Vermeulen vs Södergård')
ax.set_aspect('equal')
ax.legend()

# Vermeulen vs Zakharov
ax = axes[1]
ax.scatter(results['ft_vermeulen'], results['ft_zakharov'], alpha=0.5, s=15, c='#F18F01')
ax.plot(lims, lims, 'k--', alpha=0.5, label='Identity')
ax.set_xlabel('Vermeulen FT (nmol/L)')
ax.set_ylabel('Zakharov FT (nmol/L)')
ax.set_title('Vermeulen vs Zakharov')
ax.set_aspect('equal')
ax.legend()

# Södergård vs Zakharov
ax = axes[2]
ax.scatter(results['ft_sodergard'], results['ft_zakharov'], alpha=0.5, s=15, c='#A23B72')
ax.plot(lims, lims, 'k--', alpha=0.5, label='Identity')
ax.set_xlabel('Södergård FT (nmol/L)')
ax.set_ylabel('Zakharov FT (nmol/L)')
ax.set_title('Södergård vs Zakharov')
ax.set_aspect('equal')
ax.legend()

fig.suptitle('Pairwise Solver Comparison', fontsize=12)
plt.tight_layout()
plt.show()

### 3.3 Difference Heatmaps

In [ ]:
# Calculate percentage differences from Vermeulen (reference)
pct_diff_sodergard = 100 * (results['ft_sodergard'] - results['ft_vermeulen']) / results['ft_vermeulen']
pct_diff_zakharov = 100 * (results['ft_zakharov'] - results['ft_vermeulen']) / results['ft_vermeulen']

# Reshape for heatmap
n_tt, n_shbg = len(tt_range), len(shbg_range)
diff_sodergard_grid = pct_diff_sodergard.reshape(n_tt, n_shbg)
diff_zakharov_grid = pct_diff_zakharov.reshape(n_tt, n_shbg)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Södergård vs Vermeulen
im1 = axes[0].imshow(diff_sodergard_grid, aspect='auto', cmap='RdBu_r', 
                      origin='lower', vmin=-15, vmax=15)
axes[0].set_xticks(np.arange(0, n_shbg, 3))
axes[0].set_xticklabels([f'{shbg_range[i]:.0f}' for i in range(0, n_shbg, 3)])
axes[0].set_yticks(np.arange(0, n_tt, 5))
axes[0].set_yticklabels([f'{tt_range[i]:.0f}' for i in range(0, n_tt, 5)])
axes[0].set_xlabel('SHBG (nmol/L)')
axes[0].set_ylabel('Total Testosterone (nmol/L)')
axes[0].set_title('Södergård vs Vermeulen (% difference)')
plt.colorbar(im1, ax=axes[0], label='% difference')

# Zakharov vs Vermeulen
im2 = axes[1].imshow(diff_zakharov_grid, aspect='auto', cmap='RdBu_r', 
                      origin='lower', vmin=-15, vmax=15)
axes[1].set_xticks(np.arange(0, n_shbg, 3))
axes[1].set_xticklabels([f'{shbg_range[i]:.0f}' for i in range(0, n_shbg, 3)])
axes[1].set_yticks(np.arange(0, n_tt, 5))
axes[1].set_yticklabels([f'{tt_range[i]:.0f}' for i in range(0, n_tt, 5)])
axes[1].set_xlabel('SHBG (nmol/L)')
axes[1].set_ylabel('Total Testosterone (nmol/L)')
axes[1].set_title('Zakharov vs Vermeulen (% difference)')
plt.colorbar(im2, ax=axes[1], label='% difference')

plt.tight_layout()
plt.show()

## 4. Quantitative Comparison

### 4.1 Summary Statistics

In [ ]:
# Calculate comparison statistics
abs_diff_sodergard = results['ft_sodergard'] - results['ft_vermeulen']
abs_diff_zakharov = results['ft_zakharov'] - results['ft_vermeulen']

print("="*60)
print("SOLVER COMPARISON SUMMARY (vs Vermeulen reference)")
print("="*60)
print(f"\nGrid: TT {tt_range[0]:.0f}-{tt_range[-1]:.0f} nmol/L, SHBG {shbg_range[0]:.0f}-{shbg_range[-1]:.0f} nmol/L")
print(f"Albumin: {alb_fixed} g/L (fixed)")
print(f"N = {len(results['tt'])} comparisons")

print("\n--- Södergård vs Vermeulen ---")
print(f"Mean absolute difference: {np.mean(abs_diff_sodergard):.4f} nmol/L")
print(f"Mean % difference:        {np.mean(pct_diff_sodergard):.2f}%")
print(f"Max absolute difference:  {np.max(np.abs(abs_diff_sodergard)):.4f} nmol/L")
print(f"Correlation (r):          {np.corrcoef(results['ft_vermeulen'], results['ft_sodergard'])[0,1]:.6f}")

print("\n--- Zakharov vs Vermeulen ---")
print(f"Mean absolute difference: {np.mean(abs_diff_zakharov):.4f} nmol/L")
print(f"Mean % difference:        {np.mean(pct_diff_zakharov):.2f}%")
print(f"Max absolute difference:  {np.max(np.abs(abs_diff_zakharov)):.4f} nmol/L")
print(f"Correlation (r):          {np.corrcoef(results['ft_vermeulen'], results['ft_zakharov'])[0,1]:.6f}")

### 4.2 Example Values at Key Points

In [ ]:
# Calculate specific examples
examples = [
    {'tt': 10, 'shbg': 30, 'alb': 43, 'label': 'Low TT, Normal SHBG'},
    {'tt': 20, 'shbg': 40, 'alb': 43, 'label': 'Normal TT, Normal SHBG'},
    {'tt': 25, 'shbg': 60, 'alb': 43, 'label': 'High TT, High SHBG'},
    {'tt': 15, 'shbg': 20, 'alb': 43, 'label': 'Normal TT, Low SHBG'},
]

print("\n" + "="*80)
print("EXAMPLE CALCULATIONS")
print("="*80)
print(f"{'Scenario':<30} {'TT':>6} {'SHBG':>6} {'Vermeulen':>10} {'Södergård':>10} {'Zakharov':>10}")
print("-"*80)

for ex in examples:
    ft_v = calc_ft_vermeulen(ex['tt'], ex['shbg'], ex['alb'])
    ft_s = calc_ft_sodergard(ex['tt'], ex['shbg'], ex['alb'])
    ft_z = calc_ft_zakharov(ex['tt'], ex['shbg'], ex['alb'])
    print(f"{ex['label']:<30} {ex['tt']:>6.1f} {ex['shbg']:>6.1f} {ft_v:>10.4f} {ft_s:>10.4f} {ft_z:>10.4f}")

print("\n(All FT values in nmol/L)")

## 5. Interpretation

### Key Findings

1. **Södergård vs Vermeulen**: The Södergård solver uses higher SHBG affinity (1.2×10⁹ vs 1×10⁹ L/mol) and lower albumin affinity (2.4×10⁴ vs 3.6×10⁴ L/mol). This typically yields:
   - Slightly **lower** FT estimates at normal-high SHBG levels
   - Differences are modest (typically <5%)

2. **Zakharov vs Vermeulen**: The Zakharov allosteric model incorporates cooperative binding, where testosterone binding to one SHBG site affects binding at others. With default cooperativity (0.5):
   - FT estimates are generally **higher** than Vermeulen
   - The difference increases at higher SHBG concentrations
   - This reflects that negative cooperativity reduces effective SHBG binding

3. **Clinical Implications**:
   - All three solvers show high correlation (r > 0.99)
   - Absolute differences rarely exceed 0.1 nmol/L in normal ranges
   - For clinical decisions, solver choice matters less than measurement precision

### Recommendations

- Use **Vermeulen** as the primary reference (most widely validated)
- Consider **Zakharov** when studying SHBG biology or heterogeneous populations
- Report the method used when publishing FT estimates

In [ ]:
print("Notebook execution complete.")